In [12]:
import numpy as np
from collections import defaultdict
file = open('input19', 'r')
lines = file.read().splitlines()

In [13]:
#Calculate distance between two beacons
def distance(beaconA, beaconB):
    return sum(abs(a-b) for (a, b) in zip(beaconA, beaconB))

In [14]:
inversion = np.array([[-1,0,0],
                      [0,-1,0],
                      [0,0,-1]])

rotations = [
np.array([[ 1, 0, 0],
          [ 0, 1, 0],
          [ 0, 0, 1]]),
np.array([[ 0,-1, 0],
          [ 1, 0, 0],
          [ 0, 0, 1]]),
np.array([[-1, 0, 0],
          [ 0,-1, 0],
          [ 0, 0, 1]]),
np.array([[ 0, 1, 0],
          [-1, 0, 0],
          [ 0, 0, 1]])
] 
directions = [
np.array([[ 1, 0, 0],
          [ 0, 1, 0],
          [ 0, 0, 1]]), #z facing up
np.array([[-1, 0, 0],
          [ 0, 1, 0],
          [ 0, 0,-1]]), #z facing down
np.array([[ 0, 0,-1],
          [ 0, 1, 0],
          [ 1, 0, 0]]), #z facing right
np.array([[ 0, 0, 1],
          [ 0, 1, 0],
          [-1, 0, 0]]), #z facing left
np.array([[ 1, 0, 0],
          [ 0, 0,-1],
          [ 0, 1, 0]]), #z facing front
np.array([[ 1, 0, 0],
          [ 0, 0, 1],
          [ 0,-1, 0]])  #z facing back
]
projections = [r @ d for d in directions for r in rotations]

In [15]:
scanners = []
beacons = []
for line in lines[1:]:
    if line[:3] == '---':
        scanners.append(beacons)
        beacons = []
    elif line == '':
        continue
    else:
        beacon = np.array([int(x) for x in line.split(',')])
        beacons.append(beacon)
scanners.append(beacons)    

distances = []
for scanner in scanners:
    singleScannerDistances = []
    for a, beaconA in enumerate(scanner):
        for b, beaconB in enumerate(scanner[:a]):
            d = distance(beaconA, beaconB)
            singleScannerDistances.append((d, a, b))
    distances.append(singleScannerDistances)  

In [29]:
beaconsMap = scanners[0].copy()
usedScannerNumbers = [0]
scannersLocations = [np.array([0,0,0])]*len(scanners)

while len(usedScannerNumbers) < len(scanners):
    #Calculate distances in beacons map
    beaconsMapDistances = defaultdict(list)
    for a, beaconA in enumerate(beaconsMap):
        for b, beaconB in enumerate(beaconsMap[:a]):
            d = distance(beaconA, beaconB)
            beaconsMapDistances[d] = beaconsMapDistances[d] + [(a,b)]        
    
    for scannerNumber in list(range(1, len(scanners))):
        if scannerNumber in usedScannerNumbers:
            continue
        for projection in projections: 
            matchingBeacons = set()
            scannerOffset = None
            for scannerDistances in distances[scannerNumber]:
                sBeaconA = projection @ scanners[scannerNumber][scannerDistances[1]]
                sBeaconB = projection @ scanners[scannerNumber][scannerDistances[2]]
                sBeaconVector = sBeaconB - sBeaconA      
                
                for beaconAIndex, beaconBIndex in beaconsMapDistances[scannerDistances[0]]:    
                    mapBeaconA = beaconsMap[beaconAIndex]
                    mapBeaconB = beaconsMap[beaconBIndex]
                    mapBeaconVector = mapBeaconB - mapBeaconA                         

                    if (mapBeaconVector == sBeaconVector).all() or (mapBeaconVector == (inversion @ sBeaconVector) ).all():
                        matchingBeacons.add(tuple(mapBeaconA))
                        matchingBeacons.add(tuple(mapBeaconB))    
                        if (mapBeaconVector == sBeaconVector).all():
                            scannerOffset = sBeaconA - mapBeaconA
                        else:
                            scannerOffset = sBeaconB - mapBeaconA

            if len(matchingBeacons) >= 12:
                beaconsProjected = [(projection @ beacon) - scannerOffset for beacon in scanners[scannerNumber]]
                beaconsMap = np.append(beaconsMap, beaconsProjected, axis=0)
                beaconsMap = np.unique(beaconsMap, axis=0)            
                usedScannerNumbers.append(scannerNumber)   
                scannersLocations[scannerNumber] = -scannerOffset
                break

    print("Scanners used: " + str(len(usedScannerNumbers)) + " map size: " + str(len(beaconsMap)))

Scanners used: 2 map size: 40
Scanners used: 3 map size: 54
Scanners used: 4 map size: 68
Scanners used: 6 map size: 93
Scanners used: 7 map size: 107
Scanners used: 9 map size: 135
Scanners used: 10 map size: 149
Scanners used: 12 map size: 170
Scanners used: 13 map size: 181
Scanners used: 15 map size: 206
Scanners used: 16 map size: 220
Scanners used: 17 map size: 234
Scanners used: 18 map size: 248
Scanners used: 19 map size: 255
Scanners used: 20 map size: 269
Scanners used: 21 map size: 282
Scanners used: 23 map size: 308
Scanners used: 24 map size: 322
Scanners used: 25 map size: 336
Scanners used: 27 map size: 355
Scanners used: 28 map size: 368
Scanners used: 29 map size: 379
Scanners used: 30 map size: 390


In [28]:
max([distance(a,b) for a in scannersLocations for b in scannersLocations])

13327